In [63]:
import requests
import json
from dotenv import dotenv_values
import pandas as pd

API_KEY = dotenv_values("../collect/.env")['API_KEY']

from analyze.downloads.schedules import DICTIONARY, STOPS, ROUTES, values_to_dict
bigdf = pd.read_json('big', lines=True)
bigdf['Time'] = pd.to_datetime(bigdf['Time'])

In [58]:
from time import sleep
import requests
line = '190'
routes = ROUTES[line]
stops = [
    routes[rname][s]
    for rname in routes
    for s in routes[rname]
]
print(stops[0].keys()) # 'nr_zespolu', 'nr_przystanku'
print(STOPS[0].keys()) # 'zespol', 'slupek'
def params_from_stop(stop):
    return {
        'id': 'e923fa0e-d96c-43f9-ae6e-60518c9f3238', # schedule stuff
        'apikey': API_KEY,
        'busstopId': stop['nr_zespolu'],
        'busstopNr': stop['nr_przystanku'],
        'line': line,
    }

db_timetable_get_url = 'https://api.um.warszawa.pl/api/action/dbtimetable_get/' 
params_so_far = set()
relevant_routes = []
stops_scheds = {}
for stop in stops:
    params = params_from_stop(stop)
    if (params['busstopId'], params['busstopNr']) not in params_so_far:
        params_so_far.add((params['busstopId'], params['busstopNr']))
        print('.',end='')
        response = requests.get(db_timetable_get_url, params=params, timeout=10).json()
        print(';',end='')
        stuff = response['result']
        stuff = list(map(values_to_dict, stuff))
        stops_scheds[(params['busstopId'], params['busstopNr'])] = stuff
        relevant_routes += stuff
        # sleep(2)
    else:
        print('x',end='')
print()

relevant_routes = map(lambda x: x['trasa'], relevant_routes)
relevant_routes = set(relevant_routes)


dict_keys(['odleglosc', 'ulica_id', 'nr_zespolu', 'typ', 'nr_przystanku'])
dict_keys(['zespol', 'slupek', 'nazwa_zespolu', 'id_ulicy', 'szer_geo', 'dlug_geo', 'kierunek', 'obowiazuje_od'])
.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;x.;.;.;.;.;.;.;.;.;.;.;xxx.;xxxx.;.;.;.;xxxxxxxxx.;.;xxxxxxxxxx.;.;.;.;.;.;.;.;xxxxxx.;xxxxxxxxxxxxxxxxxxxx.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;.;x.;xxx.;xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.;xxxxxxxxxxxxx.;.;.;.;.;.;.;.;xx.;.;.;.;x.;xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.;xxxxxx


In [40]:
import pandas as pd
rel_stops = list(params_so_far)
groups, nos = list(zip(*rel_stops))

stopdf = pd.DataFrame({'group': groups, 'no': nos})
stopdf['lon'] = stopdf.apply(lambda x: [y for y in STOPS if y['zespol'] == x['group'] and y['slupek'] == x['no']][-1]['szer_geo'], axis=1)
stopdf['lat'] = stopdf.apply(lambda x: [y for y in STOPS if y['zespol'] == x['group'] and y['slupek'] == x['no']][-1]['dlug_geo'], axis=1)
stopdf['lon'] = stopdf['lon'].astype(float)
stopdf['lat'] = stopdf['lat'].astype(float)
average_lon = stopdf.groupby('group')['lon'].mean()
average_lat = stopdf.groupby('group')['lat'].mean()
stopdf = stopdf.merge(average_lon.rename('avg_lon'), left_on='group', right_index=True)
stopdf = stopdf.merge(average_lat.rename('avg_lat'), left_on='group', right_index=True)

stopdf.sort_values('group', inplace=True)
stopdf

,group,no,lon,lat,avg_lon,avg_lat
87,1003,04,52.255099,21.036174,52.254838,21.034678
48,1003,03,52.254118,21.033248,52.254838,21.034678
61,1003,02,52.255298,21.034611,52.254838,21.034678
32,1052,01,52.257087,21.040029,52.257293,21.040775
7,1052,02,52.257499,21.041521,52.257293,21.040775
...,...,...,...,...,...,...
47,7099,10,52.244544,21.001806,52.244626,21.002233
3,R-11,99,52.196605,20.922746,52.196509,20.922835
82,R-11,00,52.196412,20.922925,52.196509,20.922835
20,R-13,00,52.263590,21.047868,52.263610,21.047895


In [64]:
smdf = bigdf[bigdf['Lines'] == line]

In [80]:
brigades = smdf['Brigade'].unique()
time_min = smdf['Time'].min()
time_max = smdf['Time'].max()
print(time_min)
print(time_max)
relevant_scheds = {}

def is_in(x):
    try:
        return time_min <= pd.to_datetime('2024-02-22 '+x['czas']) <= time_max
    except Exception as e:
        # 24:01 is not inside measuring range anyway
        return False

for stop, scheds in stops_scheds.items():
    scheds = list(filter(is_in, scheds))
    relevant_scheds[stop] = scheds

relevant_scheds


2024-02-21 13:56:42
2024-02-22 14:29:05


{('5028',
  '06'): [{'symbol_2': 'null',
   'symbol_1': 'null',
   'brygada': '5',
   'kierunek': 'Znana',
   'trasa': 'TP-ZNA',
   'czas': '04:55:00'}, {'symbol_2': 'null',
   'symbol_1': 'null',
   'brygada': '12',
   'kierunek': 'Znana',
   'trasa': 'TP-ZNA',
   'czas': '05:25:00'}, {'symbol_2': 'null',
   'symbol_1': 'null',
   'brygada': '1',
   'kierunek': 'Znana',
   'trasa': 'TP-ZNA',
   'czas': '05:50:00'}, {'symbol_2': 'null',
   'symbol_1': 'null',
   'brygada': '11',
   'kierunek': 'Znana',
   'trasa': 'TP-ZNA',
   'czas': '06:10:00'}, {'symbol_2': 'null',
   'symbol_1': 'null',
   'brygada': '10',
   'kierunek': 'Znana',
   'trasa': 'TP-ZNA',
   'czas': '06:26:00'}, {'symbol_2': 'null',
   'symbol_1': 'null',
   'brygada': '9',
   'kierunek': 'Znana',
   'trasa': 'TP-ZNA',
   'czas': '06:41:00'}, {'symbol_2': 'null',
   'symbol_1': 'null',
   'brygada': '5',
   'kierunek': 'Znana',
   'trasa': 'TP-ZNA',
   'czas': '06:56:00'}, {'symbol_2': 'null',
   'symbol_1': 'null',
  

In [3]:
print(DICTIONARY['ulice'])
print(DICTIONARY['typy_przystankow'])
print(DICTIONARY['miejsca'])

{'0835': 'Gerbera', '0834': 'Gąsiorowskiego', '0837': 'Gromali', '0836': 'Głowackiego', '0831': 'Galla Anonima', '0830': 'Grzymały-Sokołowskiego', '0833': 'Gwintowa', '0838': 'Geodetów', '0547': 'Dzierzgońska', '0541': 'Dojazdowa', '0540': 'Dąbrowskiego', '0543': 'Dudziarska', '0058': 'Al.Polski Walczącej', '0053': 'rondo Jana Pawła II', '0050': 'rondo Poniatowskiego', '0057': 'rondo Hallera', '0056': 'rondo Legionów Polskich', '0610': 'Echa Leśne', '2046': 'Stefana Batorego', '2047': 'szosa lubelska', '2044': 'Statkowskiego', '2045': 'Sokola', '2042': 'Szaserów', '2040': 'Sympatyczna', '2041': 'Szpitalna', '3155': '--------- ZEGRZE PŁD. --------', '3159': '---------- KARCZEW -----------', '3158': '----------- RYNIA ------------', '3239': '----------- RUDKA ------------', '1991': 'gen.Roi', '1990': 'Pionierska', '1993': 'Przerąbkowska', '3204': '-------- STANISŁAWÓW ---------', '1627': 'Nasielska', '3202': '-------- GÓRA KALWARIA -------', '1625': 'Nowodworska', '3200': '----- WÓLKA DW

In [4]:
print(json.dumps(STOPS[0], indent=4))

{
    "zespol": "1001",
    "slupek": "01",
    "nazwa_zespolu": "Kijowska",
    "id_ulicy": "2201",
    "szer_geo": "52.248455",
    "dlug_geo": "21.044827",
    "kierunek": "al.Zieleniecka",
    "obowiazuje_od": "2023-10-21 00:00:00.0"
}


In [5]:
# def find_stop(zespol):
#     for stop in STOPS:
#         if stop['zespol'] == zespol:
#             return stop
#     return None
s = ([x for x in STOPS if x['zespol'] == '1754'])
print(len(s))
print(s[0])
print(s[-1])

2
{'zespol': '1754', 'slupek': '01', 'nazwa_zespolu': 'Kościelna', 'id_ulicy': '2068', 'szer_geo': '52.343771', 'dlug_geo': '21.250204', 'kierunek': 'Lipińska', 'obowiazuje_od': '2023-10-21 00:00:00.0'}
{'zespol': '1754', 'slupek': '02', 'nazwa_zespolu': 'Kościelna', 'id_ulicy': '0924', 'szer_geo': '52.345691', 'dlug_geo': '21.250882', 'kierunek': 'ZUS', 'obowiazuje_od': '2023-10-21 00:00:00.0'}


In [69]:

line = ROUTES['188']
route = line['TX-GWS']
##print(json.dumps(route, indent=4))

def map_stopr(stop):
    _,v = stop
    zespol = [x for x in STOPS if x['zespol'] == v['nr_zespolu'] and x['slupek'] == v['nr_przystanku']]
    zespol = zespol[-1] # todo? last "is valid from"

    return {
        'odleglosc': int(v['odleglosc']),
        'nr_zespolu': v['nr_zespolu'],
        # 'zespol': zespol,
        'lon': zespol['szer_geo'],
        'lat': zespol['dlug_geo'],
        }

r = list(map(map_stopr, route.items()))
r.sort(key=lambda x: x['odleglosc'])
print(r)

[{'odleglosc': 0, 'nr_zespolu': '4201', 'lon': '52.170675', 'lat': '20.973780'}, {'odleglosc': 335, 'nr_zespolu': '4202', 'lon': '52.168086', 'lat': '20.975185'}, {'odleglosc': 805, 'nr_zespolu': '4203', 'lon': '52.169738', 'lat': '20.979151'}, {'odleglosc': 1375, 'nr_zespolu': '4039', 'lon': '52.175245', 'lat': '20.979489'}, {'odleglosc': 2055, 'nr_zespolu': '4038', 'lon': '52.180657', 'lat': '20.978217'}, {'odleglosc': 2465, 'nr_zespolu': '4037', 'lon': '52.184076', 'lat': '20.979223'}, {'odleglosc': 3005, 'nr_zespolu': '4036', 'lon': '52.188852', 'lat': '20.980859'}, {'odleglosc': 3263, 'nr_zespolu': '4035', 'lon': '52.190923', 'lat': '20.981564'}, {'odleglosc': 3822, 'nr_zespolu': '4034', 'lon': '52.195996', 'lat': '20.983278'}, {'odleglosc': 4169, 'nr_zespolu': '4033', 'lon': '52.198991', 'lat': '20.984341'}, {'odleglosc': 4481, 'nr_zespolu': '4032', 'lon': '52.201602', 'lat': '20.985227'}, {'odleglosc': 4840, 'nr_zespolu': '4031', 'lon': '52.204780', 'lat': '20.986330'}, {'odlegl

In [70]:
def map_to_coords(stop):
    x = map_stopr(stop)
    return (float(x['lon']), float(x['lat']))

def route_to_coords(route):
    r = list(map(map_to_coords, route.items()))
    return r
print(route_to_coords(route))

[(52.22997, 21.068713), (52.235398, 21.078739), (52.240902, 21.08288), (52.243302, 21.084587), (52.222289, 21.037618), (52.224173, 21.043941), (52.226379, 21.05469), (52.228669, 21.063733), (52.245865, 21.084594), (52.250251, 21.085253), (52.170675, 20.97378), (52.169738, 20.979151), (52.168086, 20.975185), (52.180657, 20.978217), (52.175245, 20.979489), (52.188852, 20.980859), (52.184076, 20.979223), (52.195996, 20.983278), (52.190923, 20.981564), (52.240177, 21.128744), (52.240211, 21.128105), (52.201602, 20.985227), (52.198991, 20.984341), (52.210978, 20.988402), (52.20478, 20.98633), (52.216514, 20.995427), (52.21627, 20.99134), (52.216862, 21.013961), (52.217168, 21.006397), (52.219539, 21.026592), (52.217931, 21.020112), (52.24942, 21.09299), (52.250082, 21.089155), (52.24215, 21.124182), (52.244188, 21.118123), (52.244345, 21.114362), (52.245074, 21.109894), (52.246038, 21.105717), (52.247932, 21.098391)]


In [86]:
# busstopId = 7009
# busstopNr = 01
# line = 523
url = 'https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=7009&busstopNr=01&line=523&apikey=b2ec8228-6b7c-4fb9-b8ce-0c492bd6b76f'
data = requests.get(url).json()
data = data['result']
data = list(map(values_to_dict, data))
# print(json.dumps(data, indent=4))
print(json.dumps(data[0], indent = 2))

import pandas as pd
df = pd.DataFrame(data)
df['czas'] = pd.to_datetime(df['czas'])
df.sort_values(by=['trasa', 'czas'], ascending=[True, True], inplace=True)

routes = df['trasa'].unique()
print(routes)
dfs = {}
for route in routes:
    dfs[route] = df[df['trasa'] == route]

for r in routes:
    print(r, dfs[r])


{
  "symbol_2": "null",
  "symbol_1": "null",
  "brygada": "8",
  "kierunek": "PKP Olszynka Grochowska",
  "trasa": "TP-OLS",
  "czas": "04:51:00"
}
['TD-1OLSB' 'TP-OLS' 'TX-OLS' 'TX-WIA']
TD-1OLSB    symbol_2 symbol_1 brygada                 kierunek     trasa  \
9      null     null       5  PKP Olszynka Grochowska  TD-1OLSB   
13     null     null       7  PKP Olszynka Grochowska  TD-1OLSB   
22     null     null      13  PKP Olszynka Grochowska  TD-1OLSB   
24     null     null      14  PKP Olszynka Grochowska  TD-1OLSB   

                  czas  
9  2024-02-23 06:22:00  
13 2024-02-23 06:46:00  
22 2024-02-23 07:36:00  
24 2024-02-23 07:49:00  
TP-OLS     symbol_2 symbol_1 brygada                 kierunek   trasa  \
0       null     null       8  PKP Olszynka Grochowska  TP-OLS   
1       null     null      11  PKP Olszynka Grochowska  TP-OLS   
2       null     null     023  PKP Olszynka Grochowska  TP-OLS   
3       null     null     024  PKP Olszynka Grochowska  TP-OLS   
4   

C:\Users\user\AppData\Local\Temp\ipykernel_15636\3569031119.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['czas'] = pd.to_datetime(df['czas'])
